In [40]:
import json
import os
import numpy as np
gt_path='./Dataset/KITTI/training/label_2/'
gt_dicts={}
result_dicts={}
result_path='outputsgroup1/data/'

for filename in sorted(os.listdir(gt_path)):
    temp_list=[]
    with open (gt_path+filename) as f:
        filename=filename.replace(".txt","")
        for line in f:
            line=line.split()
            if line[0]!='Car':
                continue
            if (float(line[2])>1):
                continue
            temp_list.append({'class':line[0],
                  'alpha':float(line[3]),
                  'boxs':line[4:8],
                  'depth':float(line[13]),
                   'group':-1,
            '3d_attribute':line[8:15], 
            'id':filename
            })
        gt_dicts[filename]=temp_list
for filename in sorted(os.listdir(result_path)):
    temp_list=[]
    with open (result_path+filename) as f:
        filename=filename.replace(".txt","")
        for line in f:
            line=line.split()
            if line[0]!='Car':
                continue
            temp_list.append({'class':line[0],
                  'alpha':float(line[3]),
                  'boxs':line[4:8],
                  'depth':float(line[13]),
            '3d_attribute':line[8:15],
            'group':-1,
            'id':filename
            })
        result_dicts[filename]=temp_list

for key,file in gt_dicts.items():
    for index,info in enumerate(file):
        file[index]['boxs']=list(map(float,file[index]['boxs']))
        file[index]['3d_attribute']=list(map(float,file[index]['3d_attribute']))

for key,file in result_dicts.items():
    for index,info in enumerate(file):
        file[index]['boxs']=list(map(float,file[index]['boxs']))
        file[index]['3d_attribute']=list(map(float,file[index]['3d_attribute']))
def iou_2d(box1,box2):
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    area_sum = area1 + area2
    
    #計算重疊座標 設重疊box座標為 [x1,y1,x2,y2]
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x1 >= x2 or y1 >= y2:
        return 0
    else:
        inter_area = (x2-x1)*(y2-y1)
    return inter_area/(area_sum-inter_area)

In [41]:
print(len(gt_dicts))
print((result_dicts['007480'][0]['alpha'])-(result_dicts['007480'][1]['alpha']))
# if (id='000000'):
#     print(result_dicts[])
def matching (gt_label,result_label):
    for index,info in enumerate(gt_label):
        tempoverlap=0
        for result_label_index,result_label_info in enumerate(result_label): 
            overlap=iou_2d(gt_label[index]['boxs'],result_label[result_label_index]['boxs'])
            if overlap>tempoverlap and overlap>0.5:
                tempoverlap=overlap
                #print(second_frame_index)
                gt_label[index]['group']=result_label_index
                result_label[result_label_index]['group']=result_label_index



7481
-4.36


In [42]:
def depth_diff(gt_label,result_label,):
    temp_depth_list=[]
    for id,info in enumerate(result_label):
        for gt_id,gt_info in enumerate(gt_label):
            if (result_label[id]['group']==gt_label[gt_id]['group']) and result_label[id]['group']!=-1 and gt_label[gt_id]['group']!=-1:
                temp_depth_list.append(abs(result_label[id]['alpha']-gt_label[gt_id]['alpha']))
                # print(result_label[id]['alpha']-gt_label[gt_id]['alpha'])
                break
    return temp_depth_list
continue_list=[]
for filename in sorted(os.listdir(result_path)):
    filename=filename.replace(".txt","")
    matching(gt_dicts[filename],result_dicts[filename])
    depth_list=depth_diff(gt_dicts[filename],result_dicts[filename])
    continue_list+=depth_list
    
print(len(continue_list))
print(continue_list)


7413
[0.010000000000000009, 0.040000000000000036, 0.0, 0.03000000000000025, 2.71, 0.15000000000000013, 0.030000000000000027, 0.04999999999999982, 0.08000000000000007, 0.030000000000000027, 0.030000000000000027, 0.029999999999999805, 0.010000000000000009, 0.07000000000000006, 0.010000000000000009, 0.0, 0.020000000000000018, 0.020000000000000018, 0.050000000000000044, 0.07000000000000006, 0.040000000000000036, 0.050000000000000044, 0.020000000000000018, 0.040000000000000036, 0.04999999999999982, 0.010000000000000009, 0.05999999999999983, 0.07000000000000006, 0.020000000000000018, 0.020000000000000018, 0.010000000000000009, 0.010000000000000009, 0.06000000000000005, 0.050000000000000044, 3.1900000000000004, 0.030000000000000027, 0.0, 0.13000000000000012, 0.040000000000000036, 0.020000000000000018, 0.050000000000000044, 0.1100000000000001, 0.0, 0.3900000000000001, 0.010000000000000009, 0.06000000000000005, 0.020000000000000018, 0.050000000000000044, 0.0, 0.16999999999999993, 0.010000000000

In [43]:
continue_list=np.array(continue_list)
print(continue_list)

continue_list=continue_list[continue_list<3]
mean=np.mean(continue_list)
std_dev=np.std(continue_list)
new_continue_list = [x for x in continue_list if (x > mean - (2 * std_dev) and x<mean+(2*std_dev)) ]
print(len(continue_list))

print(np.mean(continue_list))
print(np.mean(new_continue_list))

[0.01 0.04 0.   ... 0.01 3.29 0.06]
7137
0.05319041614123581
0.04157256914741945


In [49]:
total_match_depth_list=np.array(continue_list)
print(np.mean(total_match_depth_list))
mean=np.mean(total_match_depth_list)
std_dev=np.std(total_match_depth_list)
new_total_match_depth_list = [x for x in total_match_depth_list if (x > mean - (2 * std_dev) and x<mean+(2*std_dev)) ]
print(mean)
print(np.mean(new_total_match_depth_list))

0.0912660334867326
0.0912660334867326
0.05398583964898285


In [44]:
img_W=1224
focal_length=7.215377000000e+02
fovx = 2 * np.arctan(img_W / (2 * focal_length))
print(fovx)

1.4068829423130154
